# Explicación de Llava de los gráficos generados por las técnicas XAI

In [2]:
pip install accelerate>=0.26.0

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Luna Santos\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:

from transformers import AutoProcessor, LlavaNextForConditionalGeneration
from PIL import Image
from accelerate import dispatch_model
import torch
import numpy as np

In [3]:
print(np.__version__)

1.21.6


## Llava para una explicación de SHAP

In [4]:
# Nombre del modelo en Hugging Face
model_name = "llava-hf/llava-v1.6-mistral-7b-hf"

# Cargar el modelo y el procesador de imágenes
processor = AutoProcessor.from_pretrained(model_name)
model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", device_map="auto", torch_dtype=torch.float16, low_cpu_mem_usage=True) 

# Imagen a analizar (gráfico)
image = Image.open("./imagenes/SHAP-V-lstm_cnn.png")





Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [5]:
# Contexto personalizado para mejorar la respuesta
context_and_question= (
    "Tenemos una imagen en la que aparecen un conjunto de 10 muestras y dos graficos para los modelos LSTM y CNN"  
    "Los graficos han sido generados por Shap, una tecnica de explicabilidad aplicada a los modelos de clasificacion"
    "Cada muestra está compuesta por secuencias de caracteres, donde cada carácter representa una operación específica." 
    "Las características extraídas representan la presencia o posición de estos caracteres en la muestra"
    "El objetivo de los modelos es clasificar correctamente cada muestra en una de varias clases (A, V, D, R) dependiendo de los patrones que contenga la muestra. "
    "El modelo predice el tipo de error: Condición de carrera (D) cuando ocurre las secuencias: no-opsr, wr, uwr, wur. DeadLock (D) cuando ocurre wdr, no-opsdr. "
    "Violación de atomicidad (A) cuando ocurre: uwdr. EL resto de combinaciones que admite la gramatica son V"
    "Shap genera un grafico que muestra que muestra cómo cada característica influye en las predicciones del modelo"
    "Eje Y: Lista de características. Las más influyentes están arriba."
    "Cada caracteristica es representada con una tupla de tres posiciones (Funcion, posicion, operacion) "
    "Donde por ejemplo ""f3-4-c"" es la caracteristica que representa la operacion ""c"", en la posicion cuarta(4), de la funcion 3 (f3)"
    "Eje X: Valores SHAP, que indican el impacto de cada característica en la predicción del modelo."
    "Valores SHAP positivos: La característica aumenta la probabilidad de que la muestra sea clasificada en esa clase."
    "Valores SHAP negativos: La característica reduce la probabilidad de que la muestra sea clasificada en esa clase."
    "Colores: Rojo: Valores altos de la característica. Azul: Valores bajos de la característica."
    "Los graficos de la imagen corresponden a la clase V"

    "Según las gráficas SHAP de la imagen, ¿qué operaciones tienen mayor impacto en la predicción de la clase V en cada modelo (LSTM y CNN)? ¿Hay diferencias en cómo cada modelo interpreta estas operaciones?"

)




In [6]:
prompt = f"<image>\nUSER: {context_and_question}\nASSISTANT:"
# Procesar la imagen y el texto

In [7]:
inputs = processor(image, prompt, return_tensors="pt", padding=True)

# Generar respuesta con opciones personalizadas
output = model.generate(
    **inputs,
    max_new_tokens=150,  # Máximo de tokens en la respuesta
    temperature=0.3,  # Control de aleatoriedad (más bajo = más preciso)
    top_p=0.9,  # Método de muestreo para respuestas variadas
    repetition_penalty=1.2,  # Evita respuestas repetitivas
)
output

c:\Users\juand\anaconda3\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\juand\anaconda3\lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1, 32000, 32000,  ..., 28739,  3118,   521]])

In [8]:
# Decodificar y mostrar la respuesta
answer = processor.batch_decode(output, skip_special_tokens=True)
print(answer)

['\nUSER: Tenemos una imagen en la que aparecen un conjunto de 10 muestras y dos graficos para los modelos LSTM y CNNLos graficos han sido generados por Shap, una tecnica de explicabilidad aplicada a los modelos de clasificacionCada muestra está compuesta por secuencias de caracteres, donde cada carácter representa una operación específica.Las características extraídas representan la presencia o posición de estos caracteres en la muestraEl objetivo de los modelos es clasificar correctamente cada muestra en una de varias clases (A, V, D, R) dependiendo de los patrones que contenga la muestra. El modelo predice el tipo de error: Condición de carrera (D) cuando ocurre las secuencias: no-opsr, wr, uwr, wur. DeadLock (D) cuando ocurre wdr, no-opsdr. Violación de atomicidad (A) cuando ocurre: uwdr. EL resto de combinaciones que admite la gramatica son VShap genera un grafico que muestra que muestra cómo cada característica influye en las predicciones del modeloEje Y: Lista de características